In [ ]:
import boll

In [ ]:
import importlib
importlib.reload(boll)

In [ ]:
import yfinance as yf

In [ ]:
from datetime import datetime

In [ ]:
# 티커를 기준으로 데이털 로드 
ticker_aapl = yf.Ticker('AAPL')

In [ ]:
df_aapl = ticker_aapl.history(
    start = '2020-01-01'
)

In [ ]:
df_aapl.index

In [ ]:
band_aapl = boll.create_band(df_aapl, _col='Close', _start='2010-01-01')

In [ ]:
band_aapl.head()

In [ ]:
trade_aapl = boll.create_trade(band_aapl)

In [ ]:
trade_aapl['trade'].value_counts()

In [ ]:
rtn_aapl, acc_rtn_aapl = boll.create_rtn(trade_aapl)

In [ ]:
acc_rtn_aapl

In [21]:
boll.create_rtn(
    boll.create_trade(
        boll.create_band(
            df_aapl, _col = 'Close', _start = '2022-01-01', 
            _cnt = 20
        )
    )
)

매수일 : 2022-01-19 00:00:00, 매수가 : 163.3712921142578
매도일 : 2022-03-24 00:00:00, 매도가 : 171.29443359375
수익율 : 1.0484977585532649
매수일 : 2022-04-26 00:00:00, 매수가 : 154.29978942871094
매도일 : 2022-07-21 00:00:00, 매도가 : 153.09754943847656
수익율 : 0.99220841457603
매수일 : 2022-08-30 00:00:00, 매수가 : 156.82345581054688
매도일 : 2022-10-25 00:00:00, 매도가 : 150.33970642089844
수익율 : 0.9586557421774888
매수일 : 2022-12-15 00:00:00, 매수가 : 134.93113708496094
매도일 : 2023-01-23 00:00:00, 매도가 : 139.4881591796875
수익율 : 1.0337729466539454
매수일 : 2023-08-04 00:00:00, 매수가 : 180.42222595214844
매도일 : 2023-08-31 00:00:00, 매도가 : 186.50308227539062
수익율 : 1.0337034768923368
매수일 : 2023-10-26 00:00:00, 매수가 : 165.6757049560547
매도일 : 2023-11-10 00:00:00, 매도가 : 185.28753662109375
수익율 : 1.1183748194718175
매수일 : 2024-01-02 00:00:00, 매수가 : 184.53208923339844
매도일 : 2024-05-03 00:00:00, 매도가 : 182.51817321777344
수익율 : 0.9890863641982736
매수일 : 2024-08-05 00:00:00, 매수가 : 208.5690155029297
매도일 : 2024-09-30 00:00:00, 매도가 : 232.48831176757812
수익

c:\ubion_2025\python\250324\boll.py:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0484977585532649' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[idx, 'rtn'] = rtn


(                 Close      center          up        down trade  rtn  \
 Date                                                                    
 2022-01-03  178.879913  172.472047  180.676457  164.267637        1.0   
 2022-01-04  176.609650  173.178682  180.091141  166.266223        1.0   
 2022-01-05  171.911835  173.362465  179.907305  166.817624        1.0   
 2022-01-06  169.042084  173.211115  180.016696  166.405533        1.0   
 2022-01-07  169.209152  173.093671  180.097547  166.089794        1.0   
 ...                ...         ...         ...         ...   ...  ...   
 2025-03-18  212.690002  232.457500  258.495777  206.419222   buy  1.0   
 2025-03-19  215.240005  230.976000  257.409379  204.542621   buy  1.0   
 2025-03-20  214.100006  229.389500  255.877849  202.901152   buy  1.0   
 2025-03-21  218.270004  228.025500  253.810140  202.240861   buy  1.0   
 2025-03-24  220.729996  226.707000  251.040841  202.373159   buy  1.0   
 
              acc_rtn  
 Date       

In [23]:
# 20일 기준 보유 내역 데이터프레임을 생성 
trade_20 = boll.create_trade(
    boll.create_band(df_aapl, _col = 'Close', _start = "2022-01-01")
)
trade_30 = boll.create_trade(
    boll.create_band(df_aapl, _col = 'Close', _start = "2022-01-01", 
                     _cnt = 30)
)

In [27]:
trade_20 = trade_20[['Close', 'trade']]

In [29]:
trade_30.drop(['center', 'up', 'down'], axis=1, inplace=True)

In [30]:
import pandas as pd

In [33]:
# trade_20, trade_30은 단순한 열 결합 
trade_df = \
    pd.concat( [trade_20, trade_30] , axis=1 , ignore_index=True)

In [35]:
# trade_df에 2 컬럼을 삭제
trade_df.drop(2, axis=1, inplace=True)

In [36]:
# 컬럼의 이름을 재지정
trade_df.columns = ['Close', 'trade_20', 'trade_30']

In [38]:
# trade_20, trade_30에 둘다 buy라는 값을 
# 가진다면 trade 컬럼에 데이터가 buy
trade_df['trade'] = ''
for idx in trade_df.index:
    if (trade_df.loc[idx, 'trade_20'] == 'buy') & \
        (trade_df.loc[idx, 'trade_30'] == 'buy'):
        trade_df.loc[idx, 'trade'] = 'buy'

In [39]:
trade_df['trade'].value_counts()

trade
       512
buy    296
Name: count, dtype: int64

In [44]:
# trade_20, trade_30에 buy가 포함되어있는가?
flag = \
    trade_df[['trade_20', 'trade_30']].isin(['buy']).all(axis=1)

In [45]:
trade_df['trade_2'] = ''

In [48]:
trade_df.loc[flag, 'trade_2'] = 'buy'

In [49]:
trade_df['trade_2'].value_counts()

trade_2
       512
buy    296
Name: count, dtype: int64

In [50]:
boll.create_rtn(trade_df)

매수일 : 2022-01-19 00:00:00, 매수가 : 163.3712921142578
매도일 : 2022-03-24 00:00:00, 매도가 : 171.29443359375
수익율 : 1.0484977585532649
매수일 : 2022-05-11 00:00:00, 매수가 : 144.37586975097656
매도일 : 2022-07-21 00:00:00, 매도가 : 153.09754943847656
수익율 : 1.0604095386752883
매수일 : 2022-09-29 00:00:00, 매수가 : 140.60916137695312
매도일 : 2022-10-25 00:00:00, 매도가 : 150.33970642089844
수익율 : 1.069202781302842
매수일 : 2022-12-15 00:00:00, 매수가 : 134.93113708496094
매도일 : 2023-01-23 00:00:00, 매도가 : 139.4881591796875
수익율 : 1.0337729466539454
매수일 : 2023-08-04 00:00:00, 매수가 : 180.42222595214844
매도일 : 2023-08-31 00:00:00, 매도가 : 186.50308227539062
수익율 : 1.0337034768923368
매수일 : 2023-10-26 00:00:00, 매수가 : 165.6757049560547
매도일 : 2023-11-10 00:00:00, 매도가 : 185.28753662109375
수익율 : 1.1183748194718175
매수일 : 2024-01-02 00:00:00, 매수가 : 184.53208923339844
매도일 : 2024-05-03 00:00:00, 매도가 : 182.51817321777344
수익율 : 0.9890863641982736
매수일 : 2025-01-16 00:00:00, 매수가 : 228.00930786132812


c:\ubion_2025\python\250324\boll.py:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0484977585532649' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[idx, 'rtn'] = rtn


(                 Close trade_20 trade_30 trade trade_2  rtn   acc_rtn
 Date                                                                 
 2022-01-03  178.879913                                  1.0  1.000000
 2022-01-04  176.609650                                  1.0  1.000000
 2022-01-05  171.911835                                  1.0  1.000000
 2022-01-06  169.042084                                  1.0  1.000000
 2022-01-07  169.209152                                  1.0  1.000000
 ...                ...      ...      ...   ...     ...  ...       ...
 2025-03-18  212.690002      buy      buy   buy     buy  1.0  1.405219
 2025-03-19  215.240005      buy      buy   buy     buy  1.0  1.405219
 2025-03-20  214.100006      buy      buy   buy     buy  1.0  1.405219
 2025-03-21  218.270004      buy      buy   buy     buy  1.0  1.405219
 2025-03-24  220.729996      buy      buy   buy     buy  1.0  1.405219
 
 [808 rows x 7 columns],
 1.4052187838464785)